In [3]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [4]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [19]:
DATA_SIZE = 80000
TRAINING_SIZE = 30000
TESTING_SIZE = 50000
DIGITS = 3
EPOCHS = 1
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

trainingOutputPath = 'd' + str(DIGITS) + 'e' + str(EPOCHS) + 's' + str((TRAINING_SIZE - TRAINING_SIZE/10)/1000)+'.csv' 

In [6]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [7]:
ctable = CharacterTable(chars)

In [8]:
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [9]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < DATA_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [10]:
print(questions[:5], expected[:5])

['5-35   ', '8-9    ', '40-155 ', '5-31   ', '69-447 '] ['-30 ', '-1  ', '-115', '-26 ', '-378']


# Processing

In [11]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [12]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:TRAINING_SIZE]
train_y = y[:TRAINING_SIZE]
test_x = x[TRAINING_SIZE:TRAINING_SIZE+TESTING_SIZE]
test_y = y[TRAINING_SIZE:TRAINING_SIZE+TESTING_SIZE]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(27000, 7, 12)
(27000, 4, 12)
Validation Data:
(3000, 7, 12)
(3000, 4, 12)
Testing Data:
(50000, 7, 12)
(50000, 4, 12)


In [69]:
# print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

# Build Model

In [13]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [24]:
training_log = list()
with open(trainingOutputPath, 'w') as output:
    output.write('loss,acc,val_loss,val_acc\n')
    for iteration in range(1):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        history_callback = model.fit(x_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=1,#EPOCHS,
                  validation_data=(x_val, y_val))
        training_log.append(history_callback.history)

        output.write(str(history_callback.history['loss'][0]) + ',')
        output.write(str(history_callback.history['acc'][0]) + ',')
        output.write(str(history_callback.history['val_loss'][0]) + ',')
        output.write(str(history_callback.history['val_acc'][0]) + '\n')
    
    
#     for i in range(10):
#         ind = np.random.randint(0, len(x_val))
#         rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
#         preds = model.predict_classes(rowx, verbose=0)
#         q = ctable.decode(rowx[0])
#         correct = ctable.decode(rowy[0])
#         guess = ctable.decode(preds[0], calc_argmax=False)
#         print('Q', q[::-1] if REVERSE else q, end=' ')
#         print('T', correct, end=' ')
#         if correct == guess:
#             print(colors.ok + '☑' + colors.close, end=' ')
#         else:
#             print(colors.fail + '☒' + colors.close, end=' ')
#         print(guess)


--------------------------------------------------
Iteration 0
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [==============================] - 24s - loss: 0.1941 - acc: 0.9475 - val_loss: 0.2518 - val_acc: 0.9143


# TESTING

In [81]:
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

Q 737-11  T 726  ☑ 726 
Q 662-70  T 592  ☑ 592 
Q 74-139  T -65  ☑ -65 
Q 355-492 T -137 ☑ -137
Q 81-496  T -415 ☑ -415
Q 312-68  T 244  ☑ 244 
Q 8-417   T -409 ☑ -409
Q 970-45  T 925  ☑ 925 
Q 422-896 T -474 ☑ -474
Q 78-673  T -595 ☑ -595
Q 224-989 T -765 ☑ -765
Q 67-300  T -233 ☑ -233
Q 603-14  T 589  ☑ 589 
Q 142-2   T 140  ☑ 140 
Q 464-245 T 219  ☑ 219 
Q 574-416 T 158  ☑ 158 
Q 921-76  T 845  ☑ 845 
Q 668-251 T 417  ☑ 417 
Q 896-330 T 566  ☑ 566 
Q 236-952 T -716 ☑ -716
Q 91-91   T 0    ☑ 0   
Q 208-528 T -320 ☑ -320
Q 313-50  T 263  ☑ 263 
Q 99-676  T -577 ☑ -577
Q 154-0   T 154  ☑ 154 
Q 667-239 T 428  ☑ 428 
Q 453-75  T 378  ☑ 378 
Q 529-52  T 477  ☑ 477 
Q 43-42   T 1    ☒ 3   
Q 860-560 T 300  ☒ 290 
Q 453-132 T 321  ☑ 321 
Q 34-15   T 19   ☑ 19  
Q 904-754 T 150  ☒ 140 
Q 390-47  T 343  ☑ 343 
Q 669-233 T 436  ☑ 436 
Q 21-71   T -50  ☑ -50 
Q 196-11  T 185  ☑ 185 
Q 711-266 T 445  ☑ 445 
Q 312-266 T 46   ☒ 56  
Q 527-3   T 524  ☑ 524 
Q 457-27  T 430  ☑ 430 
Q 73-620  T -547

Q 21-628  T -607 ☑ -607
Q 56-672  T -616 ☑ -616
Q 65-504  T -439 ☑ -439
Q 99-52   T 47   ☒ 57  
Q 1-990   T -989 ☑ -989
Q 67-687  T -620 ☑ -620
Q 185-63  T 122  ☑ 122 
Q 526-70  T 456  ☑ 456 
Q 205-646 T -441 ☑ -441
Q 1-104   T -103 ☒ -104
Q 5-988   T -983 ☑ -983
Q 5-527   T -522 ☑ -522
Q 779-288 T 491  ☒ 481 
Q 1-3     T -2   ☒ -   
Q 6-390   T -384 ☑ -384
Q 464-96  T 368  ☑ 368 
Q 420-860 T -440 ☑ -440
Q 937-72  T 865  ☑ 865 
Q 455-1   T 454  ☑ 454 
Q 44-137  T -93  ☑ -93 
Q 65-640  T -575 ☑ -575
Q 321-836 T -515 ☑ -515
Q 64-395  T -331 ☒ -332
Q 628-303 T 325  ☑ 325 
Q 185-777 T -592 ☑ -592
Q 15-736  T -721 ☑ -721
Q 628-78  T 550  ☑ 550 
Q 605-883 T -278 ☑ -278
Q 63-487  T -424 ☑ -424
Q 923-61  T 862  ☑ 862 
Q 0-190   T -190 ☑ -190
Q 192-702 T -510 ☒ -500
Q 90-313  T -223 ☑ -223
Q 898-148 T 750  ☒ 740 
Q 35-964  T -929 ☒ -930
Q 17-346  T -329 ☒ -339
Q 993-321 T 672  ☑ 672 
Q 149-2   T 147  ☑ 147 
Q 4-607   T -603 ☑ -603
Q 36-589  T -553 ☑ -553
Q 211-42  T 169  ☒ 179 
Q 5-830   T -825

Q 6-264   T -258 ☒ -267
Q 152-7   T 145  ☑ 145 
Q 924-242 T 682  ☑ 682 
Q 329-544 T -215 ☑ -215
Q 369-742 T -373 ☑ -373
Q 56-9    T 47   ☑ 47  
Q 101-972 T -871 ☑ -871
Q 42-326  T -284 ☑ -284
Q 192-720 T -528 ☑ -528
Q 57-749  T -692 ☑ -692
Q 943-78  T 865  ☑ 865 
Q 9-699   T -690 ☑ -690
Q 113-54  T 59   ☒ 69  
Q 320-655 T -335 ☑ -335
Q 729-745 T -16  ☑ -16 
Q 723-143 T 580  ☑ 580 
Q 98-16   T 82   ☑ 82  
Q 859-14  T 845  ☑ 845 
Q 838-56  T 782  ☑ 782 
Q 17-421  T -404 ☑ -404
Q 555-96  T 459  ☑ 459 
Q 3-707   T -704 ☑ -704
Q 97-53   T 44   ☑ 44  
Q 703-65  T 638  ☑ 638 
Q 84-330  T -246 ☑ -246
Q 31-137  T -106 ☒ -107
Q 3-627   T -624 ☒ -724
Q 68-580  T -512 ☑ -512
Q 538-50  T 488  ☑ 488 
Q 252-89  T 163  ☑ 163 
Q 735-202 T 533  ☑ 533 
Q 318-241 T 77   ☑ 77  
Q 153-518 T -365 ☑ -365
Q 54-530  T -476 ☑ -476
Q 959-9   T 950  ☒ 940 
Q 844-555 T 289  ☑ 289 
Q 4-38    T -34  ☑ -34 
Q 470-175 T 295  ☑ 295 
Q 813-90  T 723  ☑ 723 
Q 516-845 T -329 ☑ -329
Q 875-87  T 788  ☑ 788 
Q 94-640  T -546

Q 601-393 T 208  ☑ 208 
Q 34-596  T -562 ☑ -562
Q 509-61  T 448  ☑ 448 
Q 57-407  T -350 ☑ -350
Q 127-95  T 32   ☑ 32  
Q 39-917  T -878 ☑ -878
Q 306-357 T -51  ☑ -51 
Q 820-272 T 548  ☑ 548 
Q 78-238  T -160 ☑ -160
Q 21-505  T -484 ☑ -484
Q 426-464 T -38  ☑ -38 
Q 434-123 T 311  ☒ 301 
Q 947-378 T 569  ☑ 569 
Q 587-54  T 533  ☑ 533 
Q 388-69  T 319  ☑ 319 
Q 38-35   T 3    ☒ 2   
Q 818-397 T 421  ☑ 421 
Q 987-28  T 959  ☑ 959 
Q 51-365  T -314 ☑ -314
Q 43-134  T -91  ☑ -91 
Q 160-21  T 139  ☑ 139 
Q 44-254  T -210 ☒ -211
Q 875-4   T 871  ☒ 872 
Q 0-981   T -981 ☑ -981
Q 143-182 T -39  ☒ -49 
Q 345-335 T 10   ☑ 10  
Q 175-11  T 164  ☑ 164 
Q 568-987 T -419 ☑ -419
Q 10-48   T -38  ☑ -38 
Q 67-868  T -801 ☑ -801
Q 66-491  T -425 ☑ -425
Q 753-912 T -159 ☒ -169
Q 30-50   T -20  ☒ -10 
Q 529-760 T -231 ☑ -231
Q 317-922 T -605 ☑ -605
Q 25-761  T -736 ☑ -736
Q 639-9   T 630  ☑ 630 
Q 21-171  T -150 ☑ -150
Q 77-63   T 14   ☑ 14  
Q 10-744  T -734 ☑ -734
Q 361-270 T 91   ☑ 91  
Q 323-229 T 94  

Q 75-529  T -454 ☑ -454
Q 8-194   T -186 ☑ -186
Q 163-96  T 67   ☑ 67  
Q 73-184  T -111 ☑ -111
Q 285-421 T -136 ☑ -136
Q 360-680 T -320 ☑ -320
Q 66-844  T -778 ☑ -778
Q 58-135  T -77  ☑ -77 
Q 7-368   T -361 ☒ -371
Q 818-45  T 773  ☑ 773 
Q 62-52   T 10   ☑ 10  
Q 49-302  T -253 ☑ -253
Q 215-852 T -637 ☑ -637
Q 594-840 T -246 ☒ -247
Q 4-375   T -371 ☒ -372
Q 720-861 T -141 ☑ -141
Q 436-25  T 411  ☑ 411 
Q 761-992 T -231 ☑ -231
Q 984-98  T 886  ☑ 886 
Q 3-728   T -725 ☑ -725
Q 21-479  T -458 ☑ -458
Q 828-56  T 772  ☑ 772 
Q 758-349 T 409  ☑ 409 
Q 90-687  T -597 ☑ -597
Q 267-655 T -388 ☑ -388
Q 530-0   T 530  ☑ 530 
Q 68-572  T -504 ☑ -504
Q 63-325  T -262 ☑ -262
Q 93-180  T -87  ☒ -97 
Q 29-173  T -144 ☑ -144
Q 26-50   T -24  ☑ -24 
Q 47-289  T -242 ☑ -242
Q 186-50  T 136  ☑ 136 
Q 1-180   T -179 ☒ -189
Q 720-378 T 342  ☑ 342 
Q 78-707  T -629 ☑ -629
Q 9-454   T -445 ☑ -445
Q 150-52  T 98   ☒ 90  
Q 90-590  T -500 ☑ -500
Q 328-47  T 281  ☑ 281 
Q 78-703  T -625 ☑ -625
Q 646-61  T 585 

...

Q 343-6   T 337  ☑ 337 
Q 289-16  T 273  ☑ 273 
Q 844-51  T 793  ☑ 793 
Q 6-526   T -520 ☑ -520
Q 16-569  T -553 ☑ -553
Q 788-77  T 711  ☑ 711 
Q 89-844  T -755 ☑ -755
Q 35-15   T 20   ☒ 10  
Q 4-486   T -482 ☒ -483
Q 76-716  T -640 ☑ -640
Q 926-92  T 834  ☑ 834 
Q 499-331 T 168  ☑ 168 
Q 232-306 T -74  ☒ -64 
Q 33-647  T -614 ☑ -614
Q 660-60  T 600  ☑ 600 
Q 459-904 T -445 ☑ -445
Q 169-83  T 86   ☑ 86  
Q 235-400 T -165 ☒ -765
Q 429-59  T 370  ☑ 370 
Q 669-26  T 643  ☑ 643 
Q 871-815 T 56   ☑ 56  
Q 252-47  T 205  ☑ 205 
Q 80-494  T -414 ☑ -414
Q 41-777  T -736 ☑ -736
Q 637-8   T 629  ☒ 639 
Q 173-820 T -647 ☑ -647
Q 148-525 T -377 ☑ -377
Q 99-17   T 82   ☑ 82  
Q 8-324   T -316 ☑ -316
Q 27-679  T -652 ☑ -652
Q 793-592 T 201  ☒ 100 
Q 341-915 T -574 ☑ -574
Q 198-55  T 143  ☑ 143 
Q 313-39  T 274  ☑ 274 
Q 97-535  T -438 ☑ -438
Q 877-23  T 854  ☑ 854 
Q 907-27  T 880  ☑ 880 
Q 24-583  T -559 ☑ -559
Q 909-922 T -13  ☒ -23 
Q 641-602 T 39   ☑ 39  
Q 816-967 T -151 ☑ -151
Q 62-50   T 12  

Q 744-582 T 162  ☑ 162 
Q 833-5   T 828  ☑ 828 
Q 72-497  T -425 ☑ -425
Q 952-15  T 937  ☑ 937 
Q 19-858  T -839 ☑ -839
Q 606-563 T 43   ☑ 43  
Q 8-767   T -759 ☒ -769
Q 128-195 T -67  ☒ -77 
Q 152-1   T 151  ☒ 150 
Q 69-85   T -16  ☒ -66 
Q 828-650 T 178  ☑ 178 
Q 336-69  T 267  ☑ 267 
Q 194-7   T 187  ☑ 187 
Q 373-76  T 297  ☑ 297 
Q 837-125 T 712  ☑ 712 
Q 60-98   T -38  ☑ -38 
Q 839-12  T 827  ☑ 827 
Q 45-63   T -18  ☑ -18 
Q 76-660  T -584 ☑ -584
Q 14-332  T -318 ☑ -318
Q 24-348  T -324 ☑ -324
Q 3-392   T -389 ☒ -399
Q 413-501 T -88  ☑ -88 
Q 369-5   T 364  ☑ 364 
Q 68-676  T -608 ☑ -608
Q 998-731 T 267  ☑ 267 
Q 64-345  T -281 ☑ -281
Q 99-386  T -287 ☑ -287
Q 246-12  T 234  ☑ 234 
Q 2-834   T -832 ☑ -832
Q 266-45  T 221  ☑ 221 
Q 13-722  T -709 ☑ -709
Q 183-62  T 121  ☒ 111 
Q 719-79  T 640  ☑ 640 
Q 373-285 T 88   ☑ 88  
Q 177-253 T -76  ☑ -76 
Q 12-404  T -392 ☑ -392
Q 335-81  T 254  ☑ 254 
Q 955-85  T 870  ☑ 870 
Q 207-80  T 127  ☑ 127 
Q 54-973  T -919 ☑ -919
Q 49-141  T -92 

Q 596-46  T 550  ☑ 550 
Q 3-496   T -493 ☒ -494
Q 745-278 T 467  ☑ 467 
Q 474-949 T -475 ☑ -475
Q 166-41  T 125  ☑ 125 
Q 21-65   T -44  ☑ -44 
Q 91-931  T -840 ☑ -840
Q 993-30  T 963  ☑ 963 
Q 573-932 T -359 ☒ -369
Q 46-464  T -418 ☑ -418
Q 654-629 T 25   ☒ 36  
Q 685-371 T 314  ☑ 314 
Q 272-668 T -396 ☑ -396
Q 662-65  T 597  ☑ 597 
Q 394-63  T 331  ☑ 331 
Q 0-928   T -928 ☑ -928
Q 368-937 T -569 ☑ -569
Q 950-672 T 278  ☑ 278 
Q 21-884  T -863 ☑ -863
Q 772-24  T 748  ☑ 748 
Q 741-96  T 645  ☑ 645 
Q 366-32  T 334  ☑ 334 
Q 23-16   T 7    ☑ 7   
Q 227-60  T 167  ☑ 167 
Q 9-237   T -228 ☑ -228
Q 69-814  T -745 ☑ -745
Q 769-91  T 678  ☑ 678 
Q 85-412  T -327 ☑ -327
Q 401-50  T 351  ☑ 351 
Q 38-7    T 31   ☑ 31  
Q 676-37  T 639  ☒ 649 
Q 395-188 T 207  ☑ 207 
Q 78-33   T 45   ☑ 45  
Q 15-646  T -631 ☑ -631
Q 98-405  T -307 ☑ -307
Q 5-175   T -170 ☒ -181
Q 590-953 T -363 ☒ -263
Q 97-61   T 36   ☑ 36  
Q 861-442 T 419  ☑ 419 
Q 322-389 T -67  ☑ -67 
Q 548-4   T 544  ☑ 544 
Q 61-585  T -524

Q 76-476  T -400 ☑ -400
Q 208-384 T -176 ☒ -175
Q 209-78  T 131  ☑ 131 
Q 785-7   T 778  ☑ 778 
Q 55-34   T 21   ☑ 21  
Q 58-13   T 45   ☑ 45  
Q 652-11  T 641  ☑ 641 
Q 141-154 T -13  ☒ -23 
Q 84-226  T -142 ☑ -142
Q 785-66  T 719  ☑ 719 
Q 227-795 T -568 ☑ -568
Q 391-296 T 95   ☑ 95  
Q 592-589 T 3    ☒ 2   
Q 17-403  T -386 ☒ -385
Q 459-684 T -225 ☑ -225
Q 564-11  T 553  ☑ 553 
Q 85-243  T -158 ☑ -158
Q 188-227 T -39  ☑ -39 
Q 617-8   T 609  ☑ 609 
Q 57-212  T -155 ☑ -155
Q 94-917  T -823 ☒ -824
Q 9-139   T -130 ☑ -130
Q 650-825 T -175 ☑ -175
Q 861-4   T 857  ☑ 857 
Q 179-379 T -200 ☒ -100
Q 644-610 T 34   ☑ 34  
Q 64-5    T 59   ☑ 59  
Q 716-97  T 619  ☑ 619 
Q 945-864 T 81   ☑ 81  
Q 506-740 T -234 ☑ -234
Q 973-3   T 970  ☒ 960 
Q 81-150  T -69  ☑ -69 
Q 21-862  T -841 ☑ -841
Q 50-681  T -631 ☑ -631
Q 81-735  T -654 ☑ -654
Q 451-88  T 363  ☑ 363 
Q 82-296  T -214 ☑ -214
Q 944-57  T 887  ☑ 887 
Q 580-534 T 46   ☒ 56  
Q 815-11  T 804  ☒ 704 
Q 411-809 T -398 ☑ -398
Q 637-206 T 431 

Q 69-60   T 9    ☑ 9   
Q 188-57  T 131  ☑ 131 
Q 904-944 T -40  ☒ -30 
Q 123-390 T -267 ☒ -277
Q 49-796  T -747 ☑ -747
Q 335-77  T 258  ☑ 258 
Q 337-15  T 322  ☑ 322 
Q 257-480 T -223 ☑ -223
Q 992-19  T 973  ☑ 973 
Q 287-452 T -165 ☑ -165
Q 176-61  T 115  ☑ 115 
Q 103-63  T 40   ☑ 40  
Q 486-412 T 74   ☒ 64  
Q 260-175 T 85   ☒ 86  
Q 955-23  T 932  ☑ 932 
Q 613-560 T 53   ☒ 54  
Q 888-44  T 844  ☑ 844 
Q 539-83  T 456  ☑ 456 
Q 680-75  T 605  ☑ 605 
Q 220-92  T 128  ☑ 128 
Q 718-712 T 6    ☒ 7   
Q 754-847 T -93  ☑ -93 
Q 904-17  T 887  ☑ 887 
Q 327-987 T -660 ☒ -650
Q 77-172  T -95  ☑ -95 
Q 269-1   T 268  ☑ 268 
Q 174-208 T -34  ☑ -34 
Q 64-910  T -846 ☑ -846
Q 30-581  T -551 ☑ -551
Q 27-585  T -558 ☑ -558
Q 484-4   T 480  ☑ 480 
Q 20-273  T -253 ☑ -253
Q 690-98  T 592  ☑ 592 
Q 617-474 T 143  ☑ 143 
Q 56-324  T -268 ☑ -268
Q 461-94  T 367  ☑ 367 
Q 0-379   T -379 ☒ -389
Q 232-705 T -473 ☑ -473
Q 560-6   T 554  ☑ 554 
Q 380-907 T -527 ☑ -527
Q 44-287  T -243 ☑ -243
Q 20-88   T -68 